In [1]:
%load_ext autoreload
%autoreload 2

import numpy  as np
import pandas as pd
import h5py

import ehtplot
from matplotlib import pyplot as plt, cm

from common import dalt
from common import hallmark as hm
from common import viz

In [2]:
pf = hm.ParaFrame('cache/Illinois_thermal_w{win:d}/{mag}a{aspin:g}_i{inc:d}/avg_Rh{Rhigh:d}_{freq}.h5')

for k in set(pf.keys()) - {'path'}:
    globals()[k] = np.unique(pf[k])
    print(k, globals()[k][:16])

freq ['230GHz' '2um' '86GHz']
Rhigh [  1  10  40 160]
inc [ 10  30  50  70  90 110 130 150 170]
aspin [-0.94 -0.5   0.    0.5   0.94]
win [3 4 5]
mag ['M' 'S']


In [3]:
magmap = {'M':'MAD', 'S':'SANE'}

def readimg(f):
    with h5py.File(f) as h:
        m    = h['meta']
        meta = dalt.ImageMeta(**{k:m[k][()] for k in m.keys()})
        data = h['data'][:]
    return dalt.Image(data, meta=meta)

def plot(ax, pf):
    if len(pf) != 1:
        return
    
    if pf.freq.iloc[0] == '230GHz':
        vmax = .75e-3
    else:
        vmax = None    

    img = readimg(pf.path.iloc[0])
    viz.show(img, s=0, ax=ax, cmap='afmhot_10us', vmin=0, vmax=vmax, labels=False)
    ax.xaxis.set_ticks(np.linspace(-75,75,7))
    ax.yaxis.set_ticks(np.linspace(-75,75,7))
    ax.tick_params(color='w')
    for spine in ax.spines.values():
        spine.set_edgecolor('w')

In [4]:
f  = '230GHz'
w  = 5
a  = 0.5
Rh = 10
i  = 50

for m in ['M', 'S']:
    fig = viz.grid(pf(win=w)(mag=m)(inc=i)(freq='230GHz'), plot, aspin=[a], Rhigh=[Rh],
                   figsize=(6,6), title=f'Averaged {f} image for {magmap[m]}, inclincation $i={i}^\circ$, window {w}',
                   xtitle=r'$a_\mathrm{{spin}}={}$',  ytitle=r'$R_\mathrm{{high}}={}$',
                   xlabel=r'$x$ [$\mu$as]', ylabel=r'$y$ [$\mu$as]', 
                   fout=f'output/plot/avgsample_{magmap[m]}')
    plt.close(fig)

In [5]:
f = '230GHz'
w = 5
i = 50

for m in ['M', 'S']:
    fig = viz.grid(pf(win=w)(mag=m)(inc=i)(freq=f), plot, aspin=aspin, Rhigh=Rhigh[::-1],
                   figsize=(15,12), title=f'Averaged {f} image for {magmap[m]}, inclincation $i={i}^\circ$, window {w}',
                   xtitle=r'$a_\mathrm{{spin}}={}$',  ytitle=r'$R_\mathrm{{high}}={}$',                   
                   xlabel=r'$x$ [$\mu$as]', ylabel=r'$y$ [$\mu$as]',
                   fout=f'output/plot/avggrid_{magmap[m]}')
    plt.close(fig)